# Creating replicate weights for a survey design object (with ANES 2016 data).

In [1]:
options(jupyter.rich_display=FALSE) # Create output as usual in R

The following line loads data created with a previous *R* script or notebook.

In [2]:
load("anes-2016-prevote-desgn.RData")

The following makes use of the *survey* package. You may need to install it from
[CRAN](https://cran.r-project.org/package=survey) using the code
`install.packages("survey")` if you want to run this on your computer. (The
package is already installed on the notebook container, however.)

In [3]:
library(survey)

Loading required package: grid

Loading required package: Matrix

Loading required package: survival


Attaching package: ‘survey’


The following object is masked from ‘package:graphics’:

    dotchart




The 'automatic' type gives jackknife replicates

In [4]:
anes_2016_prevote_jk <- as.svrepdesign(anes_2016_prevote_desgn,
                                       type="auto")

The number of replicates is determined by the number of clusters

In [5]:
anes_2016_prevote_jk

Call: as.svrepdesign(anes_2016_prevote_desgn, type = "auto")
Stratified cluster jackknife (JKn) with 65 replicates.

Here we select the multistage rescaled bootstrap

In [6]:
anes_2016_prevote_boo <- as.svrepdesign(anes_2016_prevote_desgn,
                                        type="mrbbootstrap")

Warning message in mrbweights(design$cluster, design$strata, design$fpc, ...):
“Design is sampled with replacement: only first stage used”


In [7]:
anes_2016_prevote_boo

Call: as.svrepdesign(anes_2016_prevote_desgn, type = "mrbbootstrap")
Multistage rescaled bootstrap with 50 replicates.

By default the number of bootstrap replicates is 50, we can
change it to 200

In [8]:
anes_2016_prevote_boo <- as.svrepdesign(anes_2016_prevote_desgn,
                                        type="mrbbootstrap",
                                        replicates=200)

Warning message in mrbweights(design$cluster, design$strata, design$fpc, ...):
“Design is sampled with replacement: only first stage used”


In [9]:
anes_2016_prevote_boo

Call: as.svrepdesign(anes_2016_prevote_desgn, type = "mrbbootstrap", 
    replicates = 200)
Multistage rescaled bootstrap with 200 replicates.

A function to compute the percentage of 2012 Democratic and Republican voters
who vote for a candidate of the same party in 2016:

In [10]:
StayerPerc <- function(weights,data){
    tab <- xtabs(weights~vote16+recall12,data=data)
    # Remove 'inap' responses
    tab <- tab[-6,-5]
    # Column percentages
    ptab <- 100*prop.table(tab,2)
    # The diagonal are the percentages of 'stayers'
    # among the voters of 2012.
    # The first two elements of the diagonal are
    # the Democratic and Republican stayers.
    structure(diag(ptab)[1:2],
              names=c("Democratic",
                      "Republican"))
}

Estimates and replication based standard errors based on jackknife 

In [11]:
withReplicates(anes_2016_prevote_jk,
                StayerPerc)

            theta     SE
Democratic 68.765 3.8462
Republican 69.058 3.6052

In [12]:
withReplicates(anes_2016_prevote_boo,
                StayerPerc)

            theta     SE
Democratic 68.765 3.9823
Republican 69.058 3.3640